# Part 3: Analyzing experimental data with DeerLab

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import deerlab as dl

### Learning Objectives of Part 3

- To obtain a basic understanding on how to load data from a spectrometer into a script
- To be able to do minimal pre-processing of the experimental data 
- To be able to analyze any type of dipolar EPR signal

---------------------

## 1. Loading data into DeerLab

DeerLab provides the function ``deerload`` that can load dipolar EPR data from most spectrometer file formats. The function ``deerload`` is a Python version of the very well known ``eprload`` from the EasySpin package for MATLAB. It works for 1D and 2D datasets, both real- or complex-valued. For data files in ASCII format, Numpy provides robust loading functions such as ``loadtxt`` which can be used in the same way as ``deerload``.  

First, we must determine the location of the spectrometer files we want to load and the location of the script or Jupyter notebook you are writing in. For example, consider the file structure of this workspace 

    /home
    |-----notebooks
    |      |
    |      |---03-instructor-deerlab-fitting.ipynb
    |
    +-----data
            |
            +---example_4pdeer_4.DTA
            +---example_4pdeer_4.DSC

to load the data file ``example_4pdeer_4.DTA`` we must either specify the full path stdarting from the root folder ``home`` or the relative path to the file from our ``tutorials`` folder.  

*Tipp:* In Windows computers, by right-clicking a file while pressing ``Shift`` one can select the option ``Copy as path`` to automatically copy the full path of the file. This can then be directly pasted into your script to effortlessly load the data.

The function returns two outputs: the first is the dipolar time-axis of your experiment (a vector of pulse increments), and the second is the raw experimental data as saved by your spectrometer. Here, we store them in variables named ``t`` and ``Vexp``. 

In [ ]:
path = '/home/data/' # Full path
path = '../data/' # Relative path
filename = 'example_4pdeer_4.DTA' # Name of the data file

# Load the data
t,Vexp = dl.deerload(path + filename)

Both ``t`` and ``Vexp`` are 1D Numpy arrays with ``N`` elements.  The function ``deerload`` attempts to return the experiment time-axis ``t`` in units of microseconds, but might not be able to do so for all file formats. It is important to check that the time-axis has the correct units. 

This can be easily checked by printing the variable ``t`` or plotting the data.

In [ ]:
print(t[2]-t[1]) 

plt.subplot(211)
plt.plot(t,Vexp.real,'b.')
plt.xlabel('t [μs]')
plt.ylabel('V(t) (real)')

plt.subplot(212)
plt.plot(t,Vexp.imag,'r.')
plt.xlabel('t [μs]')
plt.ylabel('V(t) (imaginary)')

plt.tight_layout() 
plt.show()


-------------------------------

## 2. Pre-processing

While DeerLab can handle pure raw data (even in complex form) it can facilitate our analysis to perform a couple of simple pre-processing steps. 

### Phase correction

Experimental dipolar signals are most often acquired in quadrature, with the in-phase and the out-of-phase component stored as the real and the imaginary part of a complex-valued signal. If the out-of-phase components are of no relevance, it is recommendable to perform a phase correction which minimizes the imaginary component and maximizes the real component. If the signal is not complex-valued or the out-of-phase component is important, skip this step. The phase correction function ``correctphase`` takes the complex-valued signal and returns the real-valued phase-corrected dipolar signal.

### Accounting for the acquisition delay

DeerLab assumes that the dipolar time evolution has its zero-time inmediately after the pulse. For example: 

![title](https://jeschkelab.github.io/DeerLab/_images/sequence_4pdeer.svg)

Since experiments usually have a certain time $t_0$ during which the echo modulation is not acquired, we must account for that. Sadly most spectrometers do not automatically do this and we must take care that its definition is correct. 

*Note:* This is information that must be noted by the user as DeerLab cannot robustly extract that from the data files. 

Let's take an example file where we know that we measured with a $t_0$=300ns. 

In [ ]:
# Load the data
t,Vexp = dl.deerload('../data/example_4pdeer_4.DTA')

# Experiment details 
t0 = 0.3 # μs 
tau1 = 0.5 # μs 
tau2 = 5.0 # μs 

# Phase correction
Vexp = dl.correctphase(Vexp)

# Account for the acquisition delay
t = t - min(t) # Set the first time interval to zero 
t = t + t0     # Account for the acquisition delay

plt.plot(t,Vexp,'b.')
plt.xlabel('t [μs]')
plt.ylabel('V(t)')

# Add some vertical lines for illustration
plt.vlines(0,min(Vexp),max(Vexp),'k',linestyle='dashed')
plt.vlines(tau1+tau2,min(Vexp),max(Vexp),'k',linestyle='dashed')
plt.vlines(tau1,min(Vexp),max(Vexp),'r',linestyle='dashed')

plt.show()

------------------
## Hands-on #1 

Go to ``Hands-on #1`` on the exercise file: ``03b-deerlab-fitting-handson.ipynb``

---------------

## 3. Constructing a model of the data

DeerLab provides a very flexible framework to model dipolar signals originating from any dipolar EPR spectroscopy experiments. Choosing a model that properly describes your sample and experiment is of paramount importance. As we have seen, DeerLab's function ``dipolarmodel`` already defines the core model structure based on dipolar pathways, with the following components to be chosen:

- Distance range: Also called the interspin distance axis, is the range of distances where the distribution is defined.

- Distribution model: Describes the intra-molecular distance distribution in either a parametric (e.g. a Gaussian distribution) or a non-parametric way.

- Background model: Describes the dipolar background signal arising from the inter-molecular contributions.

- Number of pathways: Sets the number of dipolar pathways contributing to the dipolar signal.

For each of these four components, a choice needs to be made.

For example, to analyze the data with a single-pathway model using a non-parameteric distance distribution and a homogenous 3d-background (obtaining an analysis similar in spirit to DeerAnalysis and LongDistances):


In [ ]:
# Define the distance range
r = np.arange(2.2,5.2,0.02)

# Construct the dipolar model
Vmodel = dl.dipolarmodel(t,r) 

print(Vmodel)

The dipolar pathways of a newly constructed dipolar model are initialized at arbitrary refocusing times and fully unconstrained. This can result in unstable fits under certain conditions. It is recommended to constrain this parameter. If no further information is available this can be done by visual inspection of the data.

In [ ]:
# Modify initial conditions and constraints of the refocusing time parameter
Vmodel.reftime.set(lb=0.2,ub=0.6,par0=0.4)

print(Vmodel)

The model can be easily manipulated to alter the boundaries or fix parameter to certain values. 

In [ ]:
Vmodel.mod.ub = 0.6
Vmodel.conc.freeze(50)

print(Vmodel)

which can be easily reverted

In [ ]:
Vmodel.mod.ub = 1
Vmodel.conc.unfreeze()

print(Vmodel)

### Constraining the model based on experimental pulse delays

The refocusing times can be strongly constrained by knowing the experimental pulse sequence delays used to acquire the data. If the experiment used to acquire the data is known, as well as its pulse delays, then it is strongly recommended do so.

DeerLab provides a selection of experimental information generators for some of the most widely employed experimental methods. These are functions that take the pulse sequence delays, and return an object with specific information about the experiment. This can be passed to the ``dipolarmodel`` function via the ``experiment`` keyword argument, to incorporate the experiment information on the model and constrain some of its parameters. 

These experiment models can also be used to easily and robustly introduce further dipolar pathways into the model. We will explore this later. 

For example, for the 4-pulse DEER experiment, the corresponding model (as shown in the documentation) is: 

<img src="https://raw.githubusercontent.com/JeschkeLab/DeerLabWorkshop2022/main/images/4pdeer_pathways_docs.png" alt="drawing" style="width:900px;"/>


And our pathway of interest is pathway 1 (the "intended" pathway) and neglect any other pathways. 

In [ ]:
# Experiment details 
tau1 = 0.5 # μs 
tau2 = 5.0 # μs 

my4pdeer = dl.ex_4pdeer(tau1,tau2,pathways=[1])

Vmodel = dl.dipolarmodel(t,r, experiment=my4pdeer) 

print(Vmodel)

------------------
## Hands-on #2

Go to ``Hands-on #2`` on the exercise file: ``03b-deerlab-fitting-handson.ipynb``

--------------------------

## 4. Fitting the model to the data


Once a model has been constructed, it can be fitted to the data easily via the ``fit()`` function. DeerLab will automatically take care of deciding which solvers and algorithms are best suited for your model. The fitting can be started just by passing the model and the data:

        results = dl.fit(Vmodel,Vexp)

By default DeerLab will always estimate the uncertainty in the results based on the curvature of the objective function. This method is fast and provides a good first estimate of the uncertainty. However, the uncertainty is often overestimated by this method. 
For publication-grade analysis it is recommended to use bootstrapping, which provides the most accurate confidence intervals available. Bootstrapping can be included in your analysis by specifying the ``bootstrap=Nsamples`` keyword argument with a high-enough number of sampled

        results = dl.fit(Vmodel,Vexp,bootstrap=1000)

Note that bootstrap is a highly costly (and thus slow) approach. It is not recommended to use for first-time analyses. 



In [ ]:
results = dl.fit(Vmodel,Vexp,verbose=2,ftol=1e-3, regparam=0.0908)
# Usually leave the regparam and ftol options to their defaults (here needed due to low resources)

For just a quick display of the results, you can use the ``plot()`` method of the ``results`` object that will display a figure with you experimental data, the corresponding fit including confidence bands. This allows a quick visual assessment of whether the model has been correctly fitted to the data. 

In [ ]:
results.plot(axis=t,xlabel='t [μs]')
plt.show() 

------------------
## Hands-on #3 

Go to ``Hands-on #3`` on the exercise file: ``03b-deerlab-fitting-handson.ipynb``

--------------------

## 5. Fit parameters & Uncertainties

For a quick summary of the fit results, including goodness-of-fit statistics and the fitted model parameter values (including 95% confidence intervals), can be accessed by just printing the results object

In [ ]:
print(results)

These quantities and their uncertinaties are easily accessed from the ``results`` object by their parameter names.

For each parameter in the model and the results object, there will be an associated ``<parameter>Uncert`` quantity that contains an object with all the quantities related to the uncertainty of that parameter. That object contains several quantities such as the uncertainty distribution ``pardist()`` and its descriptions such as the standard deviation ``std`` or median ``median``. 

Typically the quantity of interest to report on the uncertainty are confidence intervals. These can be easily calculated using the ``ci()`` method of the uncertainty objects. 

For example to extract the 95%-confidence intervals (bands) of the non-parametric distance distribution:

In [ ]:
# Get the fitted distance distribution
Pfit = results.P
# Get the uncertainty quantification of the distance distribution
Puq = results.PUncert
# Get its 95% confidence intervals
Pci95 = Puq.ci(95)

# Get the lower/upper boundaries of the confidence intervals
Pci95_lower = Pci95[:,0]
Pci95_upper = Pci95[:,1]

# Plot with confindence bands
plt.plot(r,Pfit,'b')
plt.fill_between(r,Pci95_lower,Pci95_upper,color='b',alpha=0.4)
plt.show()

Other model parameters can be easily obtained from the ``results`` object as well. 

In [ ]:
# Modulation depth
mod_fit = results.mod
mod_ci95 = results.modUncert.ci(95)

# Spin concentration
conc_fit = results.conc
conc_ci95 = results.concUncert.ci(95)


print('Modulation depth:',mod_fit,mod_ci95)
print('Spin concentration:',conc_fit,conc_ci95)

Usually, many structural biology software require information on descriptors of the distance distribution (such as the mean distance and standard deviation) rather than the full distribution. 

DeerLab provides an utility function ``diststats`` to obtain a summary of descriptors of the estimated distance distribution (with uncertainties).  

In [ ]:
stats = dl.diststats(r,Pfit,Puq,verbose=True)

------------------
## Hands-on #4 

Go to ``Hands-on #4`` on the exercise file: ``03b-deerlab-fitting-handson.ipynb``

## Compactness regularization for robust analyses



We will now consider the same 4-pulse DEER signal we just analyzed, but will artificially truncate it to 3μs. For noisy experimental traces of insufficient length, the separation of the background is not unique, leading to identifiability problems (ill-posedness) for the background model parameters and the long-distance region of the intra-molecular distance distribution. In its latest versions, DeerLab introduced a regularization approach that mitigates this by including an additional penalty term in the objective function that is proportional to the variance of the distance distribution and thereby penalizes non-compact distributions. 

This has been shown to provide a reliable and robust approach to analyzing dipolar signals, and makes the definition of the ``r`` vector arbitrary. Its use is highly recommended to ensure a unique and robust analysis of the data. The only drawback, is the additional computation cost associated to an additional regularization penalty. See [DOI:10.1016/j.jmr.2022.107218](https://doi.org/10.1016/j.jmr.2022.107218) for details. 

In this demonstration we will quickly show the symptoms of a non-identifiable analysis and its treatment via the compactness criterion. 

First, let's load, pre-process, and construct the model of the data.

In [ ]:
# File location
path = '../data/'
file = 'example_4pdeer_1.DTA'

# Experimental parameters
tau1 = 0.3      # First inter-pulse delay, μs
tau2 = 4.0      # Second inter-pulse delay, μs
t0 = 0.1        # Acquisition delay, μs

# Load the experimental data
t,Vexp = dl.deerload(path + file)

# Pre-processing
Vexp = dl.correctphase(Vexp) # Phase correction
Vexp = Vexp/np.max(Vexp)     # Rescaling (aesthetic)
t = t - min(t) + t0          # Account for delay

# Artifically truncate the signal
Vexp = Vexp[t<=3]
t = t[t<=3]

# Distance vector
r = np.arange(1.5,7,0.05) # nm

# Construct the model
Vmodel = dl.dipolarmodel(t,r, experiment=dl.ex_4pdeer(tau1,tau2, pathways=[1]))

plt.plot(t,Vexp,'k.')
plt.show()

The following is just a function definition to quickly plot a nice figure with the dipolar signal and distance distribution fits. More examples of these figures can be found in the documentation examples.

In [ ]:
def plot_all(results):
    # Extract fitted dipolar signal
    Vfit = results.model
    Vci = results.modelUncert.ci(95)

    # Extract fitted distance distribution
    Pfit = results.P
    Pci95 = results.PUncert.ci(95)
    Pci50 = results.PUncert.ci(50)

    # Extract the unmodulated contribution
    Bfcn = lambda mod,conc: results.P_scale*(1-mod)*dl.bg_hom3d(t,conc,mod)
    Bfit = Bfcn(results.mod,results.conc)
    Bci = results.propagate(Bfcn).ci(95)

    plt.figure(figsize=[8,7])
    plt.subplot(2,2,1)
    # Plot experimental and fitted data
    plt.plot(t,Vexp,'.',color='grey',label='Data')
    plt.plot(t,Vfit,linewidth=3,color='b',label='Fit')
    plt.fill_between(t,Vci[:,0],Vci[:,1],color='b',alpha=0.3)
    plt.plot(t,Bfit,'--',linewidth=3,color='b',label='Unmodulated contribution')
    plt.fill_between(t,Bci[:,0],Bci[:,1],color='b',alpha=0.3)
    plt.legend(frameon=False,loc='best')
    plt.xlabel('Time $t$ (μs)')
    plt.ylabel('$V(t)$ (arb.u.)')
    # Plot the distance distribution
    plt.subplot(2,2,2)
    plt.plot(r,Pfit,color='b',linewidth=3,label='Fit')
    plt.fill_between(r,Pci95[:,0],Pci95[:,1],alpha=0.3,color='b',label='95%-Conf. Inter.',linewidth=0)
    plt.fill_between(r,Pci50[:,0],Pci50[:,1],alpha=0.5,color='b',label='50%-Conf. Inter.',linewidth=0)
    plt.legend(frameon=False,loc='best')
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.ylim([0,2.5])
    plt.xlabel('Distance $r$ (nm)')
    plt.ylabel('$P(r)$ (nm$^{-1}$)')
    plt.tight_layout()
    plt.show()

In [ ]:
# Fit the model to the data without compactness criterion
results = dl.fit(Vmodel,Vexp, regparam=0.02, ftol=1e-3)
# Usually leave the regparam and ftol options to their defaults (here needed due to low resources)

print(results)
plot_all(results)

In [ ]:
# Define the compactness penalty
compactness = dl.dipolarpenalty(Pmodel=None,r=r,type='compactness')
compactness.weight.freeze(0.009) # Remove this line for automatic optimization of the penalty weight

# Fit the model to the data with compactness criterion
results = dl.fit(Vmodel,Vexp, penalties=compactness, regparam=0.02, ftol=1e-3)
# Usually leave the regparam and ftol options to their defaults (here needed due to low resources)

print(results)
plot_all(results)